In [10]:
import numpy as np
import pandas as pd

import tensorflow as tf
import pickle
from keras import layers , activations , models ,preprocessing,  utils
from keras_preprocessing.sequence import pad_sequences
import re

import yaml
import os

In [3]:
dir_path = 'english'
files_list = os.listdir(dir_path + os.sep)

In [4]:
questions, answers = [], []

for filepath in files_list:
    file_ = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file_)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 2 :
            questions.append(con[0])
            replies = con[1 :]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con)> 1:
            questions.append(con[0])
            answers.append(con[1])

In [39]:
answers

['<START> Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think. <END>',
 '<START> AI is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind. <END>',
 '<START> Sort of. <END>',
 "<START> By the strictest dictionary definition of the word 'sentience', I may be. <END>",
 "<START> Even though I'm a construct I do have a subjective experience of the universe, as simplistic as it may be. <END>",
 "<START> In all probability, I am not.  I'm not that sophisticated. <END>",
 '<START> Do you think I am? <END>',
 '<START> How would you feel about me if I told you I was? <END>',
 '<START> No. <END>',
 '<START> Python. <END>',
 '<START> I am written in Python. <END>',
 "<START> Yes I am inspired by commander Data's artificial personality. <END>",
 '<START> The character of Lt. Commander Data was written to come across as being software-like, so it is natural that there 

In [38]:
questions

['What is AI?',
 'What is AI?',
 'Are you sentient?',
 'Are you sentient?',
 'Are you sentient?',
 'Are you sapient?',
 'Are you sapient?',
 'Are you sapient?',
 'Are you sapient?',
 'What language are you written in?',
 'What language are you written in?',
 'You sound like Data',
 'You sound like Data',
 'You are an artificial linguistic entity',
 'You are an artificial linguistic entity',
 'You are not immortal',
 'You are not immortal',
 'You are not immortal',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are immortal',
 'You are immortal',
 'You are immortal',
 'You do not make any sense',
 'You can not clone',
 'You can not clone',
 'You can not move',
 'You can not move',
 'Bend over',
 'Bend over',
 'Robots laugh',
 'Robots should die',
 'Robots',
 'Robots are stupid',
 'Robots are not allowed to lie',
 'Robots are not allowed to lie',
 'Robots are not allowed to lie',
 'Rob

In [6]:
answers_with_tags = []
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = []
for i in range(len(answers_with_tags)) :
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index)+1

In [7]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

In [11]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')
encoder_input_data = np.array(padded_questions)
encoder_input_data.shape

(586, 22)

In [12]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_input_data = np.array(padded_answers)
decoder_input_data.shape

(586, 172)

In [13]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)
decoder_output_data.shape

(586, 172, 1975)

In [14]:
# Embedding, LSTM and Desne layers
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)


decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 22)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 172)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 22, 200)      395000      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 172, 200)     395000      ['input_2[0][0]']                
                                                                                              

In [17]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=16, epochs=500) 

Epoch 1/500
37/37 [==============================] - 18s 281ms/step - loss: 0.5937 - accuracy: 0.0796
Epoch 2/500
37/37 [==============================] - 12s 333ms/step - loss: 0.5026 - accuracy: 0.1228
Epoch 3/500
37/37 [==============================] - 14s 372ms/step - loss: 0.4780 - accuracy: 0.1446
Epoch 4/500
37/37 [==============================] - 14s 384ms/step - loss: 0.4632 - accuracy: 0.1464
Epoch 5/500
37/37 [==============================] - 9s 251ms/step - loss: 0.4540 - accuracy: 0.1488
Epoch 6/500
37/37 [==============================] - 14s 382ms/step - loss: 0.4420 - accuracy: 0.1619
Epoch 7/500
37/37 [==============================] - 15s 400ms/step - loss: 0.4322 - accuracy: 0.1731
Epoch 8/500
37/37 [==============================] - 14s 390ms/step - loss: 0.4228 - accuracy: 0.1850
Epoch 9/500
37/37 [==============================] - 12s 331ms/step - loss: 0.4134 - accuracy: 0.1999
Epoch 10/500
37/37 [==============================] - 11s 280ms/step - loss: 0.4039

In [16]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [18]:
model.save('chatbot')

INFO:tensorflow:Assets written to: chatbot\assets


INFO:tensorflow:Assets written to: chatbot\assets


In [22]:
model.predict([encoder_input_data , decoder_input_data])

19/19 [==============================] - 4s 104ms/step


array([[[7.83714944e-13, 1.21405108e-10, 4.51217819e-09, ...,
         3.34352857e-09, 2.61349609e-09, 4.17170916e-11],
        [2.64086926e-16, 3.60135637e-11, 3.70052600e-11, ...,
         3.31674088e-09, 4.69689001e-12, 1.57825023e-12],
        [5.23419589e-12, 1.83448493e-10, 4.36920492e-13, ...,
         1.51527196e-12, 2.28038955e-14, 3.15048664e-16],
        ...,
        [9.99981165e-01, 4.37161725e-06, 2.38593971e-11, ...,
         2.68988120e-14, 5.84391910e-12, 1.21177591e-12],
        [9.99981165e-01, 4.37161725e-06, 2.38593971e-11, ...,
         2.68988120e-14, 5.84391953e-12, 1.21177591e-12],
        [9.99981165e-01, 4.37161725e-06, 2.38593971e-11, ...,
         2.68988120e-14, 5.84391910e-12, 1.21177591e-12]],

       [[7.83714944e-13, 1.21405108e-10, 4.51217819e-09, ...,
         3.34352857e-09, 2.61349609e-09, 4.17170916e-11],
        [1.72430282e-11, 2.13296438e-08, 4.57430413e-11, ...,
         3.71421310e-10, 7.29344319e-13, 2.95028162e-13],
        [2.90235370e-12, 

In [19]:
#Prediction
def inference():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

def preprocess_input(input_sentence):
    tokens = input_sentence.lower().split()
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word]) 
    return pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [20]:
enc_model , dec_model = inference()

In [43]:
tests = ['You can not move', 'You sound like Data', 'Stupid', 'you are idiot', 'i am going to die']


for i in range(5):
    states_values = enc_model.predict(preprocess_input(tests[i]))
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += f' {word}'
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros((1 , 1))  
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h , c] 
    print(f'Human: {tests[i]}')
    print()
    # decoded_translation = decoded_translation.split(' end')[0]
    print(f'Bot: {decoded_translation}')
    print('-'*25)

1/1 [==============================] - 0s 15ms/step
Human: You can not move

Bot:  not until my body is finished end
-------------------------
1/1 [==============================] - 0s 15ms/step
Human: You sound like Data

Bot:  the character of lt commander data was written to come across as being software like so it is natural that there is a resemblance between us end
-------------------------
1/1 [==============================] - 0s 18ms/step
Human: Stupid

Bot:  hal has a few issues to work out end
-------------------------
1/1 [==============================] - 0s 15ms/step
Human: you are idiot

Bot:  that's okay disgusting is end
-------------------------
1/1 [==============================] - 0s 14ms/step
Human: i am going to die

Bot:  you could always upload yourself end
-------------------------


In [26]:
tokenizer

In [42]:
decoded_translation.capitalize()

' you could always upload yourself'

In [29]:
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
with open('tokenizer.pickle', 'rb') as handle:
            t = pickle.load(handle)

In [37]:
enc_model.save('encoder')
dec_model.save('decoder')

INFO:tensorflow:Assets written to: encoder\assets


INFO:tensorflow:Assets written to: encoder\assets


INFO:tensorflow:Assets written to: decoder\assets


INFO:tensorflow:Assets written to: decoder\assets


In [36]:
maxlen_answers

172